In [1]:
import json
from typing import Sequence, List

from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.messages import HumanMessage




In [2]:
llm = OllamaFunctions(
    model='phi3',
    keep_alive=-1,
    format='json'
)

In [3]:
def add_two_numbers(number1: int, number2: int)-> int:
    """
    Add two numbers and return the result.
    
    Args:
        - number1: The first number.
        - number2: The second number.
    """
    return (number1 + number2)


def useless_tool() -> int:
    """
    This is useless tool
    """
    return "This is a useless output"



In [4]:
tools = [convert_to_openai_tool(func)['function'] for func in [
    add_two_numbers, useless_tool
]] 

In [5]:
llm = llm.bind_tools(tools=tools)

In [6]:
llm.invoke("what is the sum of 3 and 4?")

AIMessage(content='', id='run-36f88209-ac68-4d5e-af94-31de30a8be3c-0', tool_calls=[{'name': 'add_two_numbers', 'args': {'number1': 3, 'number2': 4}, 'id': 'call_86cb0b6192ed4e16980aaba783924d37'}])